## 라이브러리

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import re
import csv
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'AppleGothic'
# plt.rcParams['font.family'] = 'NanumGothic'

## vvt 파일 불러오기

In [91]:
import os
path = '../script'

# 다큐멘터리(0), 영화(1) 선택
category = 0
if category == 0:
    path = path + '/Documentary'
elif category == 1:
    path = path + '/Movie'

_list = os.listdir(path)
file_list = []
for file in _list:
    if file.startswith('.'): continue
    file_list.append(file)

file_list = sorted(file_list)
print(f'작품개수: {len(file_list)}')
# print([{i:_list[i]} for i in range(len(file_list))])
for index, title in enumerate(file_list):
    print(f'{index}: {title}')

작품개수: 18
0: 13TH
1: Bo.Burnham_.Inside
2: Bo.Burnham_.Make.Happy
3: Chasing.Coral
4: Dave.Chappelle_.Sticks.&.Stones
5: Dave.Chappelle_.Sticks._.Stones
6: David.Attenborough_.A.Life.on.Our.Planet
7: Disclosure
8: FYRE_.The.Greatest.Party.That.Never.Happened
9: Heroin(e)
10: Inside.Job
11: Kiss.the.Ground
12: Knock.Down.The.House
13: Seaspiracy
14: The.Social.Dilemma
15: Three.Identical.Strangers
16: Virunga
17: What.the.Health


## 여기서 작품 선택합니다 꼭!!

In [92]:
# 바로 위 코드에서 file_list를 참고하여 작품 선택
title = file_list[10]

file_name = title + '.WEBRip.Netflix.en[cc]'
print(f'{path}/{title}/{file_name}')

../script/Documentary/Inside.Job/Inside.Job.WEBRip.Netflix.en[cc]


## VTT 파일 읽기

In [93]:
opened_file = open(f'{path}/{title}/{file_name}.vtt', encoding='utf8')
content = opened_file.read()
segments = content.split('\n\n')    # 한줄씩 정리

In [94]:
segments[:5]

['WEBVTT',
 'NOTE Netflix\nNOTE Profile: webvtt-lssdh-ios8\nNOTE Date: 2016/11/22 22:40:24',
 'NOTE SegmentIndex\nNOTE Segment=598.181 45880@644 282\nNOTE Segment=597.263 59760@46524 364\nNOTE Segment=597.222 62191@106284 378\nNOTE Segment=597.263 52609@168475 317\nNOTE Segment=592.968 60907@221084 369\nNOTE Segment=595.136 58499@281991 353\nNOTE Segment=595.553 65295@340490 396\nNOTE Segment=597.848 49345@405785 299\nNOTE Segment=596.888 65472@455130 396\nNOTE Segment=596.804 62734@520602 381\nNOTE Segment=544.127 44333@583336 269\nNOTE /SegmentIndex',
 '                                                                                 ',
 '                    \n1\n00:01:08.652 --> 00:01:10.946  position:23.33%,start  align:start size:27.14%  line:79.29% \n<c.white><c.mono_sans>NARRATOR:</c.mono_sans></c.white>']

## 불필요한 부분 제거

In [95]:
m = re.compile(r"\<.*?\>")
def clean(content):
    if '♪' in content: return None
    if '[' in content: return None
    if ']' in content: return None
    if '(' in content: return None
    if ')' in content: return None
    if '{' in content: return None
    if '}' in content: return None
    new_content = m.sub('',content)
    new_content = new_content.replace('-->','')
    remove_point_1 = new_content.find('position:')
    remove_point_2 = new_content.find('% \n')
    new_content = new_content[:remove_point_1] + new_content[remove_point_2+3:]
    return new_content

In [96]:
new_segments = [clean(s) for s in segments if len(s)!=0][4:]
new_segments[:5]

['                    \n1\n00:01:08.652  00:01:10.946  NARRATOR:',
 '2\n00:01:08.652  00:01:10.946  Iceland is a stable democracy',
 '3\n00:01:10.988  00:01:12.656  with a high standard',
 '4\n00:01:10.988  00:01:12.656  of living,',
 '5\n00:01:12.698  00:01:14.324  and, until recently,']

## 줄번호 / 속도 시작 / 속도 끝 / 자막 - 데이터 분리

In [97]:
trimmed_segments = []
for segment in new_segments:
    if segment != None and segment.strip() != '':
      split_segment = segment.split()
      times = split_segment[:3]
      script = ' '.join(split_segment[3:])
      times.append(script)
      trimmed_segments.append(times)

In [98]:
trimmed_segments[:5]

[['1', '00:01:08.652', '00:01:10.946', 'NARRATOR:'],
 ['2', '00:01:08.652', '00:01:10.946', 'Iceland is a stable democracy'],
 ['3', '00:01:10.988', '00:01:12.656', 'with a high standard'],
 ['4', '00:01:10.988', '00:01:12.656', 'of living,'],
 ['5', '00:01:12.698', '00:01:14.324', 'and, until recently,']]

## 데이터 저장

In [99]:
with open(f'{path}/{title}/{file_name}.csv', 'w', encoding='utf8', newline='') as f:
    for line in trimmed_segments:
        thewriter = csv.writer(f)
        thewriter.writerow(line)

## 최종 - 줄번호 / 속도 시작 / 속도 끝 / 대사 별로 컬럼 분리

In [100]:
pd.set_option('display.max_colwidth', None)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv', names=['num', 'start', 'end', 'script'])

display(len(data), data.tail())


3783

,num,start,end,script
3778,3800,01:48:17.783,01:48:21.412,And damn these friends
3779,3801,01:48:21.454,01:48:25.540,That keep on combing back
3780,3802,01:48:21.454,01:48:25.540,Their smiles
3781,3803,01:48:25.582,01:48:29.253,I save my grace
3782,3804,01:48:25.582,01:48:29.253,With half-assed guilt


In [101]:
if data['script'].iloc[-1][-1] not in ['.', '!', '?']:
    data['script'].iloc[-1] = data['script'].iloc[-1] + '.'

data.tail()

,num,start,end,script
3778,3800,01:48:17.783,01:48:21.412,And damn these friends
3779,3801,01:48:21.454,01:48:25.540,That keep on combing back
3780,3802,01:48:21.454,01:48:25.540,Their smiles
3781,3803,01:48:25.582,01:48:29.253,I save my grace
3782,3804,01:48:25.582,01:48:29.253,With half-assed guilt.


In [102]:
data['script'] = data['script'].str.replace('\"', '')

# 쪼개진 문장 합치기
for index, current_row in data.iterrows():
  if current_row['script'][-1] in ['.', '!', '?']:
    continue
  else:
    row_index = index + 1
    while True:
      next_row = data.loc[row_index]
      new_script = data.loc[index, 'script'] + ' ' + next_row['script']
      data.loc[index, 'script'] = new_script
      if next_row['script'][-1] in ['.', '!', '?']:
        data.loc[index, 'end'] = next_row['end']
        break
      else:
        row_index += 1
print(row_index)
# 필요없는 토막문장들 제거
data = data.drop_duplicates(subset='end')
data.head(30)

3782


,num,start,end,script
0,1,00:01:08.652,00:01:17.661,"NARRATOR: Iceland is a stable democracy with a high standard of living, and, until recently, extremely low unemployment and government debt."
7,8,00:01:19.538,00:01:23.458,We had the complete infrastructure of a modern society.
10,11,00:01:23.500,00:01:26.336,"Clean energy, food production."
12,13,00:01:26.378,00:01:30.131,Fisheries with a quota system to manage them.
14,15,00:01:30.173,00:01:33.594,"Good healthcare, education, you know, clean air."
16,17,00:01:33.635,00:01:35.303,Not much crime.
17,18,00:01:35.345,00:01:37.765,It's a good place for families to live.
19,20,00:01:37.806,00:01:41.643,We had almost end-of-history status.
21,22,00:01:41.685,00:01:54.281,"NARRATOR: But in 2000, Iceland's government began a broad policy of deregulation that would have disastrous consequences, first for the environment and then for the economy."
30,31,00:01:54.322,00:02:06.126,They started by allowing multinational corporations like Alcoa to build giant aluminum-smelting plants and exploit Iceland's geothermal and hydroelectric energy sources.


In [103]:
data.to_csv(f'{path}/{title}/{file_name}.csv', columns=['start', 'end', 'script'], index=False)
data = pd.read_csv(f'{path}/{title}/{file_name}.csv')

data.head()

,start,end,script
0,00:01:08.652,00:01:17.661,"NARRATOR: Iceland is a stable democracy with a high standard of living, and, until recently, extremely low unemployment and government debt."
1,00:01:19.538,00:01:23.458,We had the complete infrastructure of a modern society.
2,00:01:23.500,00:01:26.336,"Clean energy, food production."
3,00:01:26.378,00:01:30.131,Fisheries with a quota system to manage them.
4,00:01:30.173,00:01:33.594,"Good healthcare, education, you know, clean air."
